In [23]:
import re
import pandas as pd

def pauseLogFile(fname = 'x.log'): 
    data = {}
    with open(fname, 'r') as f:
        record = False
        def filter_lines(line):
            global record
            if 'lusearch starting =====' in line:
                record = True
            return record
        text = '\n'.join(filter(filter_lines, f))
        lines = re.findall(r'- \[.*?ms\] .*', text)
        lastTime = 0
        lastStage = None
        for line in lines:
            currTime = int(float(re.match(r'- \[(.*?)ms\]', line).group(1)) * 1000000)
            if 'Safepoint start' in line:
                currStage = 'Safepoint start'
            elif 'Stop mutators done' in line:
                currStage = 'Stop mutators done'
            else:
                currStage = re.match(r'.* Activate (\w+)', line) 
                currStage = currStage.group(1) if currStage and currStage.group(1) != 'Final' else None
            if lastStage is not None:
                time = currTime - lastTime
                if lastStage not in data: data[lastStage] = []
                data[lastStage].append(time)
            lastStage = currStage
            lastTime = currTime
    for k in data:
        print(f'--- {k} ---')
        print(pd.DataFrame({k: data[k]}).describe())

pauseLogFile()

--- Safepoint start ---
       Safepoint start
count      1839.000000
mean      46698.336052
std       10232.085110
min       11602.000000
25%       42755.000000
50%       45816.000000
75%       49188.000000
max      224013.000000
--- Stop mutators done ---
       Stop mutators done
count        1.839000e+03
mean         2.344932e+05
std          1.118620e+05
min          4.932300e+04
25%          1.990305e+05
50%          2.423280e+05
75%          2.651055e+05
max          4.296628e+06
--- Initial ---
            Initial
count  1.839000e+03
mean   5.638580e+05
std    1.291985e+05
min    2.821530e+05
25%    4.895035e+05
50%    5.479440e+05
75%    6.108625e+05
max    2.731444e+06
--- Release ---
            Release
count  1.839000e+03
mean   2.106178e+05
std    9.344299e+04
min    4.878200e+04
25%    2.015250e+05
50%    2.109980e+05
75%    2.209170e+05
max    3.854203e+06
--- Prepare ---
            Prepare
count  1.980000e+02
mean   1.273249e+05
std    2.180117e+05
min    2.565800e+04
